### Problema o Aplicación:
La Universidad Autónoma de Occidente (UAO) quiere mejorar la experiencia de sus estudiantes al recibir recomendaciones o comentarios sobre la institución. Para lograrlo, la UAO quiere desarrollar un sistema de procesamiento de lenguaje natural (PLN) que analice las recomendaciones de los estudiantes y clasifique automáticamente el sentimiento expresado en ellas, asignando una calificación de 0 a 5 estrellas. Esto permitirá a la UAO identificar rápidamente las áreas de mejora y responder eficazmente a las preocupaciones de los estudiantes.

### Planteamiento:
**Datos:** Utilizaremos un conjunto de datos que contenga recomendaciones o comentarios que contenga labels* con las caracterizticas de su clasificación de 0 a 5 donde cero es un comentario negativo y 5 es un comentario positico o de 5 estrellas. 
**Modelo de PLN:** Utilizaremos un modelo pre-entrenado de PLN para clasificación de texto. Fine-tunearemos** este modelo para que pueda comprender y clasificar específicamente las recomendaciones sobre la UAO.  
**Entrenamiento y Evaluación:** Entrenaremos el modelo con un conjunto de datos etiquetado que contenga recomendaciones y calificaciones en estrellas. Luego, evaluaremos el rendimiento del modelo para asegurarnos de que pueda clasificar correctamente el sentimiento expresado en las recomendaciones.

### Implementación:
1. **Carga de Datos:** Cargaremos un conjunto de datos que contenga recomendaciones sobre la UAO junto con las calificaciones en estrellas.
2. **Preparación de Datos:** Preprocesaremos los datos, tokenizando las recomendaciones y convirtiendo las calificaciones en estrellas a un formato adecuado para el modelo.
3. **Modelo y Fine-Tuning:** Utilizaremos un modelo pre-entrenado de PLN, como BERT, y lo fine-tunearemos en nuestro conjunto de datos de recomendaciones de la UAO.
4. **Entrenamiento y Evaluación:** Entrenaremos el modelo con los datos de entrenamiento y evaluaremos su rendimiento utilizando métricas como precisión
5. **Uso del Modelo:** Una vez entrenado y evaluado, el modelo estará listo para recibir nuevas recomendaciones sobre la UAO y clasificarlas automáticamente en una escala de 0 a 5 estrellas según el sentimiento expresado en ellas. (para el ejercicio plantado se realizara todo en ingles, debido al set de datos para re entena al igual que el modelo pre entrenado)


In [0]:
# Instalar las bibliotecas necesarias para trabajar con PLN
# transformers: Biblioteca de Hugging Face para utilizar modelos pre-entrenados de PLN y realizar fine-tuning
# evaluate: Herramientas para evaluar modelos de PLN
# datasets: Conjuntos de datos listos para usar en tareas de PLN
# accelerate: Biblioteca para acelerar el entrenamiento de modelos de PLN
%pip install transformers evaluate datasets accelerate -q


In [0]:
# Reiniciar el entorno de Python para aplicar cambios en las bibliotecas instaladas
dbutils.library.restartPython()

### 1 - Librerias y Datasets

[Repositorio de Datasets](https://huggingface.co/datasets)

In [0]:
# Importar las funciones necesarias de la biblioteca datasets para trabajar con conjuntos de datos
from datasets import list_datasets, load_dataset, DatasetInfo

# Importar las funciones necesarias de la biblioteca transformers para trabajar con modelos de PLN y entrenamiento
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Importar las funciones necesarias de la biblioteca huggingface_hub para acceder a datasets y autenticación
from huggingface_hub import list_datasets, dataset_info
from huggingface_hub import notebook_login

# Cargar el dataset "yelp_review_full"
dataset = load_dataset("yelp_review_full")

# Mostrar el dataset cargado
dataset


### 2 - Muestra dato y estructura de entrenamiento

In [0]:
# Acceder a una instancia específica del conjunto de entrenamiento del dataset
dataset["train"][80]

In [0]:
# Para mejorar el entrenamiento, seleccionar un subconjunto más pequeño de datos
# Seleccionar aleatoriamente 1200 instancias del conjunto de entrenamiento
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(1200))

# Seleccionar aleatoriamente 300 instancias del conjunto de evaluación
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(300))


### 3 - Tokenizer

In [0]:
# Modelo a utilizar (BERT base cased)
modelo = "bert-base-cased"

# Cargar el tokenizador correspondiente al modelo
tokenizer = AutoTokenizer.from_pretrained(modelo)

# Función para tokenizar los ejemplos
def tokenize_function(examples):
    """
    Función para tokenizar los ejemplos del dataset.
    Args:
        examples (dict): Un diccionario que contiene los ejemplos del dataset, donde cada clave es una característica del ejemplo.
    Returns:
        dict: Un diccionario con las características tokenizadas.
    """
    # Tokenizar las secuencias de texto utilizando el tokenizador
    # Aplicar relleno para que todas las secuencias tengan la misma longitud y truncamiento para limitar las secuencias a la longitud máxima permitida por el modelo
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Tokenizar el conjunto de entrenamiento
small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)

# Tokenizar el conjunto de evaluación
small_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)


### 4 - Modelo

In [0]:
# Cargamos el modelo pre-entrenado para clasificación de secuencias y especificamos el número de etiquetas de clasificación

model = AutoModelForSequenceClassification.from_pretrained(modelo, num_labels=5)

### 5 - Entrenamiento

In [0]:
import numpy as np
import evaluate

# Cargar la métrica de accuracy
metric = evaluate.load("accuracy")

# Función para calcular las métricas
def compute_metrics(eval_pred):
    """
    Calcula la métrica de accuracy para las predicciones.
    Args:
        eval_pred (tuple): Una tupla que contiene los logits y las etiquetas reales.
    Returns:
        float: El valor de la métrica de accuracy.
    """
    # Extraer los logits y las etiquetas reales
    logits, labels = eval_pred
    
    # Obtener las predicciones
    predictions = np.argmax(logits, axis=-1)
    
    # Calcular el accuracy
    return metric.compute(predictions=predictions, references=labels)


In [0]:
# Hacemos login en Hugging Face Model Hub para guardr el modelo mas adelante
notebook_login()

In [0]:
# Definimos los argumentos del entrenamiento
training_args = TrainingArguments(
    'Tallerfn_tun_mod_eval',            # Nombre del directorio donde se guardarán los checkpoints y logs del entrenamiento
    evaluation_strategy="steps",         # Estrategia de evaluación: evaluar cada "n" pasos
    logging_steps=10,                    # Intervalo para imprimir logs durante el entrenamiento
    num_train_epochs=2,                  # Número de epochs de entrenamiento
    push_to_hub=True,                    # Subir los checkpoints del modelo a Hugging Face Hub
)


[Argumentos Training](https://huggingface.co/transformers/v4.2.2/main_classes/trainer.html)

In [0]:
# Instanciamos el objeto Trainer con todo lo que hemos preparado
trainer = Trainer(
    model=model,                           # Modelo a entrenar
    args=training_args,                    # Argumentos de entrenamiento definidos previamente
    train_dataset=small_train_dataset,     # Conjunto de datos de entrenamiento
    eval_dataset=small_eval_dataset,       # Conjunto de datos de evaluación
    compute_metrics=compute_metrics,       # Función para calcular las métricas
)

Entrenamos...

In [0]:
# 7 min con 80 ejemplos (aprox)
trainer.train()

In [0]:
# Evaluamos el modelo
results = trainer.evaluate()
print(results)

#### Guardar el modelo en local

In [0]:
# Aplica si queremos correr el modelo en entorno controlado
trainer.save_model("./modelo")

#### Subir a HuggingFace

In [0]:
trainer.push_to_hub("edchaud/Tallerfn_tun_mod_eval")

In [0]:
tokenizer.push_to_hub("edchaud/Tallerfn_tun_mod_eval")

## Conclusiones Técnicas:

1. **Selección del Modelo de PLN:** Utilizamos un modelo pre-entrenado de PLN, como BERT, debido a su capacidad para capturar el contexto y las relaciones semánticas en el texto. BERT es particularmente adecuado para tareas de clasificación de texto como esta.

2. **Fine-Tuning del Modelo:** Fine-tuneamos el modelo BERT en nuestro conjunto de datos etiquetado para adaptarlo específicamente a la tarea de clasificación de sentimientos en las recomendaciones sobre la UAO. Esto nos permite aprovechar el conocimiento preexistente del modelo y adaptarlo a nuestras necesidades específicas.

3. **Tokenización de los Datos:** Utilizamos el tokenizador correspondiente al modelo BERT para tokenizar las recomendaciones y convertirlas en un formato adecuado para el modelo. Aplicamos relleno y truncamiento para estandarizar la longitud de las secuencias de texto.

4. **Entrenamiento y Evaluación del Modelo:** Entrenamos el modelo con el conjunto de datos de entrenamiento y evaluamos su rendimiento utilizando métricas como la precisión. La precisión nos proporciona una medida de qué tan bien el modelo puede clasificar correctamente el sentimiento expresado en las recomendaciones.

## Conclusiones Generales:

1. **Mejora de la Experiencia del Estudiante:** La implementación de este sistema de PLN permitirá a la UAO mejorar la experiencia de sus estudiantes al analizar automáticamente las recomendaciones y clasificar el sentimiento expresado en ellas. Esto les permitirá identificar rápidamente las áreas de mejora y responder de manera eficaz a las preocupaciones de los estudiantes.

2. **Automatización del Proceso:** La automatización del proceso de análisis y clasificación de recomendaciones permite a la UAO ahorrar tiempo y recursos, ya que no es necesario revisar manualmente cada recomendación. El sistema puede procesar grandes cantidades de comentarios de manera eficiente.

3. **Escalabilidad:** Una vez que el modelo esté entrenado y evaluado, estará listo para recibir nuevas recomendaciones sobre la UAO y clasificarlas automáticamente en una escala de 0 a 5 estrellas según el sentimiento expresado en ellas. Esto proporciona una solución escalable que puede adaptarse a las necesidades futuras de la institución.

4. **Feedback Continuo:** La implementación de este sistema proporciona un mecanismo para recopilar feedback continuo de los estudiantes, lo que permite a la UAO realizar mejoras proactivas en sus servicios y procesos en función de las necesidades y preferencias de la comunidad estudiantil.
